In [1]:
import numpy as np
import torch.nn as nn

import random
import torch

from torch.utils.data import DataLoader, TensorDataset
from IPython.display import clear_output
from tqdm import tqdm
from data import*

In [2]:
# We define an architecture where we introduce a Quantum State the model retuns an escalar that tell us if the state is entangled or not

class EntanglementNN(nn.Module):
    def __init__(self):
        super(EntanglementNN, self).__init__()

        # Convolutional layers to extract features
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, padding=1)   # Output: 4 x 4 x 4
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, padding=1)   # Output: 8 x 4 x 4
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)  # Output: 16 x 4 x 4

        # Batch Normalization layers for better convergence
        self.bn1 = nn.BatchNorm2d(4)
        self.bn2 = nn.BatchNorm2d(8)
        self.bn3 = nn.BatchNorm2d(16)

        # Fully connected layers to process the flattened features
        self.fc1 = nn.Linear(16 * 4 * 4, 32)  # First fully connected layer
        self.fc2 = nn.Linear(32, 1)           # Final output layer (scalar)

        # Dropout layer to reduce overfitting
        self.dropout = nn.Dropout(0.5)

        # Activation functions
        self.relu = nn.ReLU()        # ReLU activation for hidden layers
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for output (binary classification)
    
    def forward(self, x):
        # Forward pass through the convolutional layers with BatchNorm and ReLU activation
        x = self.relu(self.bn1(self.conv1(x)))  # Output: 4 x 4 x 4
        x = self.relu(self.bn2(self.conv2(x)))  # Output: 8 x 4 x 4
        x = self.relu(self.bn3(self.conv3(x)))  # Output: 16 x 4 x 4

        # Flatten the feature maps before passing into the fully connected layers
        x = x.view(x.size(0), -1)  # Output: (batch_size, 16 * 4 * 4)

        # Forward pass through the fully connected layers
        x = self.dropout(self.relu(self.fc1(x)))  # Output: 32
        x = self.fc2(x)                           # Output: 1 (scalar)

        # Apply the sigmoid activation to get the probability output
        x = self.sigmoid(x)

        return x

# Instantiate the model
model = EntanglementNN()


In [3]:
# We generates the data for train our model, for this, we specify the data parameters
#   N_train the number of total data generated for an epoch
#   entanglement_criterion to specify the criterion used to differenciate between entangled and separate states
#   N_epochs the number of epochs for train our model
#   batch_size to define number of data used for each epoch to update the weights

N_train     = 10000
criterion   = "ppt"
N_epochs    = 30
batch_size  = 32

In [4]:
# Define the criterion to implement the loss function

criterion = nn.BCELoss()

In [5]:
# Define a function with different parameters thats train our model

def train_model(model, criterion, entanglement_criterion, optimizer, epochs = 20):
    # Trains a PyTorch model using the provided data loader, criterion (loss function), and optimizer.

    # Parameters:
    # model (torch.nn.Module): The neural network model to train.
    # train_loader (torch.utils.data.DataLoader): DataLoader containing the training data.
    # criterion (torch.nn.Module): The loss function to minimize.
    # optimizer (torch.optim.Optimizer): The optimizer used to update the model's weights.
    # epochs (int): The number of epochs to train the model.
    
    model.train()  # Set the model in training mode
    epoch_loss = None

    for epoch in range(epochs):

        clear_output(wait = True)

        state_train, p_train = create_dataset(num_samples = N_train, criterion = entanglement_criterion)
        train_dataset   = TensorDataset(state_train, p_train)
        train_loader    = DataLoader(train_dataset, batch_size=32)

        running_loss = 0.0
        
        if epoch_loss != None:
            print(f"Previous average Loss: {epoch_loss:.4f}")

        # Use tqdm for progress display
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch [{epoch + 1}/{epochs}]")

            for inputs, labels in tepoch:
                # Reset the gradients
                optimizer.zero_grad()

                # Forward pass: get model predictions
                outputs = model(inputs)

                # Compute the loss
                loss = criterion(outputs, labels)

                # Backward pass and optimization step
                loss.backward()
                optimizer.step()

                # Accumulate the running loss
                running_loss += loss.item()

                # Update tqdm bar with loss
                tepoch.set_postfix(loss=loss.item())

        # Print the average loss for this epoch
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch + 1}/{epochs}]")
        

In [ ]:
# Start the train process and save the model

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
train_model(model, criterion, "ppt", optimizer, epochs = N_epochs)
torch.save(model.state_dict(), "model.pth")

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
train_model(model, criterion, "ppt", optimizer, epochs = N_epochs)
torch.save(model.state_dict(), "model.pth")

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)
train_model(model, criterion, "ppt", optimizer, epochs = 10*N_epochs)
torch.save(model.state_dict(), "model.pth")